In [3]:
import pandas as pd
import hashlib as hl
import os

In [7]:
#constants
directory = "C://Users//RAGS//Desktop//Dev_Env//test_notes"

In [2]:
def hash_file(filename):
   """"This function returns the SHA-1 hash
   of the file passed into it"""

   # make a hash object
   h = hl.sha1()

   # open file for reading in binary mode
   with open(filename,'rb') as file:

       # loop till the end of the file
       chunk = 0
       while chunk != b'':
           # read only 1024 bytes at a time
           chunk = file.read(1024)
           h.update(chunk)

   # return the hex representation of digest
   return h.hexdigest()

In [19]:
def list_files(dir_path):
    file_paths = []
    for root, directories, files in os.walk(dir_path):
        for filename in files:
            filepath = os.path.join(root, filename)
            file_paths.append(filepath)
    return file_paths

In [73]:
def create_hash_file(filename):
    files = list_files(directory)
    hash = []
    for i in files:
        hash.append(hash_file(i))
    #hash
    df = pd.DataFrame(files, columns=['filenames'])
    df['hash'] = hash
    df.to_csv(filename)

In [74]:
create_hash_file('old_hash.csv')


In [49]:
create_hash_file('new_hash.csv')

In [70]:
df1 = pd.read_csv('old_hash.csv')
df2 = pd.read_csv('new_hash.csv')
df=pd.concat([df1,df2])
df.drop(0,axis=1,inplace=True)
df
#df.to_excel('final.xlsx')
# df.drop_duplicates(keep=False, inplace=True)
# df

,Unnamed: 0,filenames,hash
1,1,C://Users//RAGS//Desktop//Dev_Env//test_notes\...,2eb2505c120943759d7cd62108233035a9675ea0
2,2,C://Users//RAGS//Desktop//Dev_Env//test_notes\...,7a2d467caf12585e116665fb512d96588f35197f
3,3,C://Users//RAGS//Desktop//Dev_Env//test_notes\...,cec1478fec8b31fc8d7a55cd0fa1ab0167e6f711
4,4,C://Users//RAGS//Desktop//Dev_Env//test_notes\...,06eb1807e5a8baba377e4361801580af0f67378c
5,5,C://Users//RAGS//Desktop//Dev_Env//test_notes\...,96d0b698ed797e61ce1c298d23a8de3c6f672385
...,...,...,...
57,57,C://Users//RAGS//Desktop//Dev_Env//test_notes\...,971036e0dc800d45428a34cece4662d7df25379c
58,58,C://Users//RAGS//Desktop//Dev_Env//test_notes\...,ef9d592ff983892c37f10b71f7413492c302bf27
59,59,C://Users//RAGS//Desktop//Dev_Env//test_notes\...,637140a7362a893643c821f1a00f3b8c99bb4e52
60,60,C://Users//RAGS//Desktop//Dev_Env//test_notes\...,02f90d59b867a2907a748643337898fe3d47b62e


In [ ]:
from __future__ import print_function

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

import google.auth
from googleapiclient.http import MediaFileUpload

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/drive']


def main():
    """Shows basic usage of the Drive v3 API.
    Prints the names and ids of the first 10 files the user has access to.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
        
    # create drive api client
    service = build('drive', 'v3', credentials=creds)

    try:

        file_metadata = {'name': 'download.jpeg'}
        media = MediaFileUpload('download.jpeg',
                                mimetype='image/jpeg')
        # pylint: disable=maybe-no-member
        file = service.files().create(body=file_metadata, media_body=media,
                                      fields='id').execute()
        print(F'File ID: {file.get("id")}')

    except HttpError as error:
        print(F'An error occurred: {error}')
        file = None


    try:

        # Call the Drive v3 API
        results = service.files().list(
            pageSize=10, fields="nextPageToken, files(id, name)").execute()
        items = results.get('files', [])

        if not items:
            print('No files found.')
            return
        print('Files:')
        for item in items:
            print(u'{0} ({1})'.format(item['name'], item['id']))
    except HttpError as error:
        # TODO(developer) - Handle errors from drive API.
        print(f'An error occurred: {error}')


if __name__ == '__main__':
    main()